## Download data

In [75]:
!wget -O CNV.gz https://tcga-xena-hub.s3.us-east-1.amazonaws.com/download/TCGA.OV.sampleMap\%2FGistic2_CopyNumber_Gistic2_all_data_by_genes.gz
!wget -O Methyl.gz https://tcga-xena-hub.s3.us-east-1.amazonaws.com/download/TCGA.OV.sampleMap\%2FHumanMethylation27.gz
!wget -O mRNA.gz https://tcga-xena-hub.s3.us-east-1.amazonaws.com/download/TCGA.OV.sampleMap\%2FHT_HG-U133A.gz
!wget -O RNAseq.gz https://tcga-xena-hub.s3.us-east-1.amazonaws.com/download/TCGA.OV.sampleMap%2FHiSeqV2_PANCAN.gz
!wget -O gdc_methyl.gz https://gdc-hub.s3.us-east-1.amazonaws.com/download/TCGA-OV.methylation27.tsv.gz

--2021-10-17 06:35:19--  https://tcga-xena-hub.s3.us-east-1.amazonaws.com/download/TCGA.OV.sampleMap%2FGistic2_CopyNumber_Gistic2_all_data_by_genes.gz
Resolving tcga-xena-hub.s3.us-east-1.amazonaws.com (tcga-xena-hub.s3.us-east-1.amazonaws.com)... 52.216.229.38
Connecting to tcga-xena-hub.s3.us-east-1.amazonaws.com (tcga-xena-hub.s3.us-east-1.amazonaws.com)|52.216.229.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1633969 (1.6M) [binary/octet-stream]
Saving to: ‘CNV.gz’

CNV.gz              100%[===================>]   1.56M  1.20MB/s    in 1.3s    

2021-10-17 06:35:22 (1.20 MB/s) - ‘CNV.gz’ saved [1633969/1633969]

--2021-10-17 06:35:22--  https://tcga-xena-hub.s3.us-east-1.amazonaws.com/download/TCGA.OV.sampleMap%2FHumanMethylation27.gz
Resolving tcga-xena-hub.s3.us-east-1.amazonaws.com (tcga-xena-hub.s3.us-east-1.amazonaws.com)... 52.216.229.38
Connecting to tcga-xena-hub.s3.us-east-1.amazonaws.com (tcga-xena-hub.s3.us-east-1.amazonaws.com)|52.216.229.

## Read data into DataFrame

In [76]:
!gunzip *.gz

In [78]:
import pandas as pd

df_list = {}
data_cnv_file = "CNV"
data_mRNA_file = "mRNA"
data_RNAseq_file = "RNAseq"
data_methyl_file = "Methyl"
data_gdc_methyl_file = "gdc_methyl"

df_list["cnv"] = pd.read_csv(data_cnv_file, sep="\t", index_col=0)
df_list["mrna"] = pd.read_csv(data_mRNA_file, sep="\t", index_col=0)
df_list["rnaseq"] = pd.read_csv(data_RNAseq_file, sep="\t", index_col=0)
df_list["methyl"] = pd.read_csv(data_methyl_file, sep="\t", index_col=0)
df_list["gdc_methyl"] = pd.read_csv(data_gdc_methyl_file, sep="\t", index_col=0)

for key in df_list.keys():
    print(f"{key} \t {df_list[key].shape}")

cnv 	 (24776, 579)
mrna 	 (12042, 593)
rnaseq 	 (20530, 308)
methyl 	 (27578, 616)
gdc_methyl 	 (27578, 613)


In [ ]:
df_biomed_list = {}
df_biomed_list['aliquot'] = pd.read_csv('../csv_data/aliquot.csv')
df_biomed_list['analyte'] = pd.read_csv('../csv_data/analyte.csv')
df_biomed_list['diagnostic_slides'] = pd.read_csv('../csv_data/diag_slides.csv')
df_biomed_list['portion'] = pd.read_csv('../csv_data/portion.csv')
df_biomed_list['sample'] = pd.read_csv('../csv_data/sample.csv')
df_biomed_list['shipment_portion'] = pd.read_csv('../csv_data/shipment_portion.csv')
df_biomed_list['slide'] = pd.read_csv('../csv_data/slide.csv')
df_biomed_list['ssf_normal_controls'] = pd.read_csv('../csv_data/ssf_norm.csv')
df_biomed_list['ssf_tumor_samples'] = pd.read_csv('../csv_data/ssf_tumor.csv')

df_clinic_list = {}
df_clinic_list['drug'] = pd.read_csv('../csv_data/drug.csv')
df_clinic_list['v1_nte'] = pd.read_csv('../csv_data/v1_nte.csv')
df_clinic_list['v1'] = pd.read_csv('../csv_data/v1.csv')
df_clinic_list['nte'] = pd.read_csv('../csv_data/nte.csv')
df_clinic_list['omf_v4'] = pd.read_csv('../csv_data/omf_v4.csv')
df_clinic_list['patient'] = pd.read_csv('../csv_data/patient.csv')
df_clinic_list['radiation'] = pd.read_csv('../csv_data/radiation.csv')

for key in df_biomed_list.keys():
    print(f"{key} \t {df_biomed_list[key].shape}")

for key in df_clinic_list.keys():
    print(f"{key} \t {df_clinic_list[key].shape}")


## Intersections

### Intersection \#1: CNV + mRNA + DNA methylation

In [79]:
col_idx = pd.concat([df_list["cnv"], df_list["mrna"], df_list["methyl"]], axis=0, join="inner").columns
print(len(col_idx))

555


In [ ]:
df_final_biomed = pd.concat(df_biomed_list).fillna(0)
df_final_biomed.to_csv('../biomed_clinic_data/biomedical_data.csv', index=False)

df_final_clinic = pd.concat(df_clinic_list).fillna(0)
df_final_clinic.to_csv('../biomed_clinic_data/clinical_data.csv', index=False)

df_final_biomed_clinic = pd.concat([df_final_biomed, df_final_clinic]).fillna(0)
df_final_biomed_clinic.to_csv('../biomed_clinic_data/biomedical_clinical_data.csv', index=False)

In [80]:
df_list["cnv_1"] = df_list["cnv"][col_idx]
df_list["mrna_1"] = df_list["mrna"][col_idx]
df_list["methyl_1"] = df_list["methyl"][col_idx]

### Intersection \#2: CNV + RNAseq + DNA methylation

In [81]:
col_idx = pd.concat([df_list["cnv"], df_list["rnaseq"], df_list["methyl"]], axis=0, join="inner").columns
print(len(col_idx))

292


In [82]:
df_list["cnv_2"] = df_list["cnv"][col_idx]
df_list["rnaseq_2"] = df_list["rnaseq"][col_idx]
df_list["methyl_2"] = df_list["methyl"][col_idx]

## Zero/Missing/NA values removal

In [84]:
def remove_missing_zero_data(df):
    df = df.dropna(axis=0, how='any')
    df = df[df.where(df != 0).any(axis=1)]
    return df

In [85]:
import numpy as np

for key in df_list.keys():
    df_list[key] = remove_missing_zero_data(df_list[key])
    print(f"{key} \t {df_list[key].shape}")

cnv 	 (24776, 579)
mrna 	 (12042, 593)
rnaseq 	 (20318, 308)
methyl 	 (21666, 616)
gdc_methyl 	 (21675, 613)
cnv_1 	 (24776, 555)
mrna_1 	 (12042, 555)
methyl_1 	 (21694, 555)
cnv_2 	 (24776, 292)
rnaseq_2 	 (20318, 292)
methyl_2 	 (22528, 292)


## Normalisation

In [86]:
def min_max_normalization(df):
    for column in df.columns:
        df[column] = (df[column] - df[column].min())/ (df[column].max() - df[column].min())
    return df

In [87]:
for key in df_list.keys():
    df_list[key] = min_max_normalization(df_list[key])

## Concatenation for multi-omics data

In [88]:
final_list = {}

final_list["mrna_methyl"] = pd.concat([df_list["mrna_1"], df_list["methyl_1"]], axis=0, join="inner")
final_list["methyl_cnv"] = pd.concat([df_list["cnv_1"], df_list["methyl_1"]], axis=0, join="inner")
final_list["cnv_mrna"] = pd.concat([df_list["cnv_1"], df_list["mrna_1"]], axis=0, join="inner")

final_list["methyl_rnaseq"] = pd.concat([df_list["methyl_2"], df_list["rnaseq_2"]], axis=0, join="inner")
final_list["cnv_rnaseq"] = pd.concat([df_list["cnv_2"], df_list["rnaseq_2"]], axis=0, join="inner")

final_list["cnv_methyl_mrna"] = pd.concat([final_list["methyl_cnv"], df_list["mrna_1"]], axis=0, join="inner")
final_list["cnv_methyl_rnaseq"] = pd.concat([final_list["methyl_rnaseq"], df_list["cnv_2"]], axis=0, join="inner")


for key in final_list.keys():
    print(f"{key} \t {final_list[key].shape}")

mrna_methyl 	 (33736, 555)
methyl_cnv 	 (46470, 555)
cnv_mrna 	 (36818, 555)
methyl_rnaseq 	 (42846, 292)
cnv_rnaseq 	 (45094, 292)
cnv_methyl_mrna 	 (58512, 555)
cnv_methyl_rnaseq 	 (67622, 292)


In [89]:
for key in final_list.keys():
    final_list[key].to_csv(f'{key}.csv')

In [90]:
for key in df_list.keys():
    df_list[key].to_csv(f'{key}.csv')